In [1]:
def matrix(sequenceA, sequenceB, match_fn, gap_A_fn, 
                                gap_B_fn, penalize_end_gaps, align_globally, 
                                 score_only)
"""Generate a score and traceback matrix according to Needleman-Wunsch 
   
      This implementation allows the usage of general gap functions and is rather 
      slow. It is automatically called if you define your own gap functions. You 
      can force the usage of this method with ``force_generic=True``. 
      """ 
      # Create the score and traceback matrices. These should be in the 
      # shape: 
      # sequenceA (down) x sequenceB (across) 
      lenA, lenB = len(sequenceA), len(sequenceB) 
      score_matrix, trace_matrix = [], [] 
      for i in range(lenA + 1): 
          score_matrix.append([None] * (lenB + 1)) 
          if not score_only: 
              trace_matrix.append([None] * (lenB + 1)) 
   
      # Initialize first row and column with gap scores. This is like opening up 
      # i gaps at the beginning of sequence A or B. 
      for i in range(lenA + 1): 
          if penalize_end_gaps[1]:  # [1]:gap in sequence B 
              score = gap_B_fn(0, i) 
          else: 
              score = 0 
          score_matrix[i][0] = score 
      for i in range(lenB + 1): 
          if penalize_end_gaps[0]:  # [0]:gap in sequence A 
              score = gap_A_fn(0, i) 
          else: 
              score = 0 
          score_matrix[0][i] = score 
485   
486      # Fill in the score matrix.  Each position in the matrix 
487      # represents an alignment between a character from sequence A to 
488      # one in sequence B.  As I iterate through the matrix, find the 
489      # alignment by choose the best of: 
490      #    1) extending a previous alignment without gaps 
491      #    2) adding a gap in sequenceA 
492      #    3) adding a gap in sequenceB 
493      for row in range(1, lenA + 1): 
494          for col in range(1, lenB + 1): 
495              # First, calculate the score that would occur by extending 
496              # the alignment without gaps. 
497              nogap_score = score_matrix[row - 1][col - 1] + \ 
498                  match_fn(sequenceA[row - 1], sequenceB[col - 1]) 
499   
500              # Try to find a better score by opening gaps in sequenceA. 
501              # Do this by checking alignments from each column in the row. 
502              # Each column represents a different character to align from, 
503              # and thus a different length gap. 
504              # Although the gap function does not distinguish between opening 
505              # and extending a gap, we distinguish them for the backtrace. 
506              if not penalize_end_gaps[0] and row == lenA: 
507                  row_open = score_matrix[row][col - 1] 
508                  row_extend = max([score_matrix[row][x] for x in range(col)]) 
509              else: 
510                  row_open = score_matrix[row][col - 1] + gap_A_fn(row, 1) 
511                  row_extend = max([score_matrix[row][x] + gap_A_fn(row, col - x) 
512                                    for x in range(col)]) 
513   
514              # Try to find a better score by opening gaps in sequenceB. 
515              if not penalize_end_gaps[1] and col == lenB: 
516                  col_open = score_matrix[row - 1][col] 
517                  col_extend = max([score_matrix[x][col] for x in range(row)]) 
518              else: 
519                  col_open = score_matrix[row - 1][col] + gap_B_fn(col, 1) 
520                  col_extend = max([score_matrix[x][col] + gap_B_fn(col, row - x) 
521                                    for x in range(row)]) 
522   
523              best_score = max(nogap_score, row_open, row_extend, col_open, 
524                               col_extend) 
525              if not align_globally and best_score < 0: 
526                  score_matrix[row][col] = 0 
527              else: 
528                  score_matrix[row][col] = best_score 
529   
530              # The backtrace is encoded binary. See _make_score_matrix_fast 
531              # for details. 
532              if not score_only: 
533                  trace_score = 0 
534                  if rint(nogap_score) == rint(best_score): 
535                      trace_score += 2 
536                  if rint(row_open) == rint(best_score): 
537                      trace_score += 1 
538                  if rint(row_extend) == rint(best_score): 
539                      trace_score += 8 
540                  if rint(col_open) == rint(best_score): 
541                      trace_score += 4 
542                  if rint(col_extend) == rint(best_score): 
543                      trace_score += 16 
544                  trace_matrix[row][col] = trace_score 
545   
546      return score_matrix, trace_matrix 

SyntaxError: invalid syntax (<ipython-input-1-e22f7b8d5963>, line 2)